In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyClassifier
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

C:\Users\berna\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\berna\anaconda3\envs\teachopencadd\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Set path to this notebook
HERE = Path(_dh[-1])
levels_up = 2
HDAC1= HERE.parents[levels_up-1]/'input'
HDAC1

WindowsPath('C:/Users/berna/Documents/Projects/BAP_2021/teachopencadd-master-latest/teachopencadd/talktorials/HDAC-89dan/BioactivityModels/HDAC1/input')

In [3]:
df1 = pd.read_pickle(HDAC1/"HDAC1_1024B.csv")
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pChEMBL_HDAC1
0,CHEMBL4579774,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, ...","[1546832, 196319, 548421, 1456686, 1474683, 40...",898.32,17.00,63.00,12.00,5.00,8.30,5.00,1.00,2.00,6.07
1,CHEMBL3655934,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[17966510, 2478511, 10872982, 33025828, 119321...",304.12,6.00,22.00,7.00,5.00,0.19,2.00,0.00,0.00,6.33
2,CHEMBL4103874,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[5296414, 3821889, 137380, 1835646, 7332227, 6...",468.20,11.00,34.00,7.00,3.00,4.21,2.00,0.00,1.00,6.54
3,CHEMBL4173137,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[2620858, 1112196, 137380, 1288122, 28861, 132...",377.15,6.00,28.00,3.00,3.00,4.23,3.00,0.00,0.00,5.92
4,CHEMBL97387,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7048323, 2421530, 3771845, 37564616, 2140073,...",332.08,5.00,23.00,4.00,2.00,2.03,2.00,0.00,0.00,6.23


In [4]:
# Add column for activity
df1["activity"] = np.zeros(len(df1))

# Mark every molecule as active with an pIC50 of >= 6.6, 0 otherwise
df1.loc[df1[df1.pchembl >= 6.6].index, "activity"] = 1.0

# NBVAL_CHECK_OUTPUT
print("Number of active compounds:", int(df1.activity.sum()))
print("Number of inactive compounds:", len(df1) - int(df1.activity.sum()))

Number of active compounds: 2362
Number of inactive compounds: 2130


In [5]:
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl,activity
0,CHEMBL3655935,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[569161, 2478511, 10872982, 15618770, 11932100...",292.07,4.00,20.00,5.00,3.00,2.17,2.00,0.00,0.00,6.73,1.00
1,CHEMBL3655945,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[35600637, 2478511, 10872982, 16070626, 119321...",326.03,5.00,21.00,5.00,3.00,2.56,2.00,0.00,0.00,5.84,0.00
2,CHEMBL3339031,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, ...","[1705, 13925431, 5032800, 4181618, 729515, 693...",514.22,11.00,36.00,8.00,4.00,5.35,3.00,1.00,1.00,8.27,1.00
3,CHEMBL2018448,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 8033062, 2817337, 33707642,...",378.17,5.00,28.00,5.00,3.00,1.65,3.00,0.00,1.00,5.40,0.00
4,CHEMBL4062908,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1705, 16948577, 10242645, 7125875, 338217, 11...",441.18,11.00,31.00,7.00,4.00,4.78,3.00,0.00,0.00,6.59,0.00


#### By using MACCS fingerprints that are 166 bits 

In [6]:
X1 = np.array(list((df1['fp_MACCS']))).astype(float)
#X.shape
Y1 = df1["activity"].values
#y.shape

In [7]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train1.shape)
print("Shape of test data:", X_test1.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 167)
Shape of test data: (899, 167)


In [8]:
# Defines and builds the LazyClassifier
clf1 =  LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
model1,predictions1 = clf1.fit(X_train1, X_test1, Y_train1, Y_test1)

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████      | 28/29 [00:09<00:00,  3.26it/s]

[16:50:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  2.96it/s]


In [9]:
predictions1.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.82,0.82,0.82,0.82,0.20
RandomForestClassifier,0.82,0.82,0.82,0.82,0.32
LGBMClassifier,0.82,0.82,0.82,0.82,0.18
ExtraTreesClassifier,0.82,0.81,0.81,0.82,0.34
NuSVC,0.81,0.81,0.81,0.81,1.51
SVC,0.80,0.80,0.80,0.80,1.36
BaggingClassifier,0.80,0.80,0.80,0.80,0.17
KNeighborsClassifier,0.79,0.79,0.79,0.79,0.45
ExtraTreeClassifier,0.78,0.77,0.77,0.78,0.02


In [10]:
predictions1.to_csv('LazyPredict_MACCS.csv')

### USING 1024 BITS FINGERPRINTS

In [11]:
#By using Morgan fingerprints with radius of 3 and 1024 bits
X2 = np.array(list((df1['fp_Morgan3']))).astype(float)
#X.shape
Y2 = df1["activity"].values
#y.shape

In [12]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train2.shape)
print("Shape of test data:", X_test2.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1024)
Shape of test data: (899, 1024)


In [13]:
clf2 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models2, predictions2 = clf2.fit(X_train2, X_test2, Y_train2, Y_test2)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:40<00:08,  2.94s/it]

[16:51:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:41<00:00,  1.43s/it]


In [14]:
predictions2.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.85,0.84,0.84,0.85,1.12
RandomForestClassifier,0.84,0.83,0.83,0.84,0.92
NuSVC,0.82,0.82,0.82,0.82,9.11
LGBMClassifier,0.82,0.82,0.82,0.82,0.36
XGBClassifier,0.82,0.81,0.81,0.82,1.10
SVC,0.82,0.81,0.81,0.82,8.20
BaggingClassifier,0.82,0.81,0.81,0.82,1.38
QuadraticDiscriminantAnalysis,0.80,0.80,0.80,0.80,1.07
KNeighborsClassifier,0.79,0.79,0.79,0.79,2.86


In [15]:
predictions2.to_csv('LazyPredict_Morgan3.csv')

In [16]:
#By using Morgan fingerprints with radius of 3 and 1024 bits and feautures
X3 = np.array(list((df1['fp_MorganF']))).astype(float)
#X.shape
Y3 = df1["activity"].values
#y.shape

In [17]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, Y3, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train3.shape)
print("Shape of test data:", X_test3.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1024)
Shape of test data: (899, 1024)


In [18]:
clf3 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models3, predictions3 = clf3.fit(X_train3, X_test3, Y_train3, Y_test3)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:40<00:08,  2.97s/it]

[16:52:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:42<00:00,  1.45s/it]


In [19]:
predictions3.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.83,0.83,0.83,0.83,1.15
RandomForestClassifier,0.83,0.83,0.83,0.83,0.92
XGBClassifier,0.82,0.82,0.82,0.82,1.64
NuSVC,0.82,0.82,0.82,0.82,8.99
SVC,0.82,0.82,0.82,0.82,8.04
LGBMClassifier,0.82,0.81,0.81,0.82,0.43
BaggingClassifier,0.80,0.80,0.80,0.80,1.27
QuadraticDiscriminantAnalysis,0.79,0.79,0.79,0.79,1.08
DecisionTreeClassifier,0.79,0.79,0.79,0.79,0.38


In [20]:
predictions3.to_csv('LazyPredict_MorganF.csv')

In [21]:
#By using MAP4 fingerprints with 1024 bits
X4 = np.array(list((df1['fp_MAP4']))).astype(float)
#X.shape
Y4 = df1["activity"].values
#y.shape

In [22]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, Y4, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train4.shape)
print("Shape of test data:", X_test4.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1024)
Shape of test data: (899, 1024)


In [23]:
# Defines and builds the lazyclassifier
clf4 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models4, predictions4 = clf4.fit(X_train4, X_test4, Y_train4, Y_test4)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:07<00:09,  3.29s/it]

[16:53:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:12<00:00,  2.50s/it]


In [24]:
predictions4.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.82,0.82,0.82,0.82,4.04
XGBClassifier,0.82,0.82,0.82,0.82,2.86
LGBMClassifier,0.82,0.82,0.82,0.82,2.70
ExtraTreesClassifier,0.81,0.81,0.81,0.81,1.18
KNeighborsClassifier,0.81,0.80,0.80,0.80,2.98
NuSVC,0.79,0.79,0.79,0.79,9.41
SVC,0.79,0.78,0.78,0.78,8.48
BaggingClassifier,0.77,0.78,0.78,0.77,13.88
QuadraticDiscriminantAnalysis,0.76,0.76,0.76,0.76,1.05


In [25]:
predictions4.to_csv('LazyPredict_MAP4.csv')

## USE 2048 BITS FINGERPRINTS

In [26]:
df2 = pd.read_pickle(HDAC1/"HDAC1_2048B.csv")
df2 = df2.sample(frac=1).reset_index(drop=True)
df2

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl
0,CHEMBL2023527,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[371106, 106027, 1055378, 943140, 1666919, 191...",799.31,13.00,58.00,16.00,7.00,2.79,4.00,1.00,3.00,5.34
1,CHEMBL3585975,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, ...","[2612565, 10511, 5141245, 12778580, 7332227, 3...",398.15,9.00,28.00,5.00,2.00,5.10,3.00,0.00,0.00,5.38
2,CHEMBL3287021,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 1798302, 28570269, 4298697, 3063821...",326.05,7.00,21.00,7.00,3.00,1.67,2.00,0.00,0.00,6.20
3,CHEMBL3287264,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 11837907, 2172950, 218190, 8864663,...",380.10,8.00,25.00,7.00,3.00,3.10,2.00,0.00,0.00,6.77
4,CHEMBL3689802,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9461852, 4568935, 7998790, 5513169, 9830328, ...",302.17,7.00,22.00,5.00,3.00,2.05,2.00,0.00,0.00,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487,CHEMBL2011672,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5456017, 756812, 2475943, 4368278, 5668350, 1...",416.19,11.00,30.00,5.00,3.00,2.51,1.00,0.00,0.00,4.32
4488,CHEMBL241757,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25259155, 3346815, 2314293, 11956864, 1856549...",376.10,4.00,27.00,6.00,2.00,3.34,4.00,0.00,0.00,6.70
4489,CHEMBL481955,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[15212094, 14728336, 9368997, 4066765, 2140073...",444.97,7.00,26.00,6.00,1.00,5.09,2.00,0.00,0.00,5.46
4490,CHEMBL3287262,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 11837907, 9564045, 218190, 6876230,...",416.10,9.00,28.00,7.00,3.00,3.26,3.00,0.00,0.00,7.23


In [27]:
# Add column for activity
df2["activity"] = np.zeros(len(df2))

# Mark every molecule as active with an pIC50 of >= 6.6, 0 otherwise
df2.loc[df2[df2.pchembl >= 6.6].index, "activity"] = 1.0

# NBVAL_CHECK_OUTPUT
print("Number of active compounds:", int(df2.activity.sum()))
print("Number of inactive compounds:", len(df2) - int(df2.activity.sum()))

Number of active compounds: 2362
Number of inactive compounds: 2130


In [28]:
#2048 bits Morgan fingerprints with radius of 3
X5 = np.array(list((df2['fp_Morgan3']))).astype(float)
#X.shape
Y5 = df2["activity"].values
#y.shape

In [29]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5, Y5, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train5.shape)
print("Shape of test data:", X_test5.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2048)
Shape of test data: (899, 2048)


In [30]:
clf5 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models5, predictions5 = clf5.fit(X_train5, X_test5, Y_train5, Y_test5)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:27<00:19,  6.58s/it]

[16:54:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:30<00:00,  3.12s/it]


In [31]:
predictions5.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.87,0.87,0.87,0.87,1.85
RandomForestClassifier,0.86,0.85,0.85,0.86,1.46
LGBMClassifier,0.84,0.84,0.84,0.84,0.64
XGBClassifier,0.83,0.83,0.83,0.83,1.94
BaggingClassifier,0.83,0.83,0.83,0.83,3.01
SVC,0.82,0.82,0.82,0.82,17.97
NuSVC,0.82,0.82,0.82,0.82,19.74
DecisionTreeClassifier,0.80,0.80,0.80,0.80,0.84
CalibratedClassifierCV,0.80,0.80,0.80,0.80,17.16


In [32]:
predictions5.to_csv('LazyPredict_Morgan3_2048.csv')

In [33]:
#2048 bits Morgan fingerprints with feautures
X6 = np.array(list((df2['fp_MorganF']))).astype(float)
#X.shape
Y6 = df2["activity"].values
#y.shape

In [34]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train6, X_test6, Y_train6, Y_test6 = train_test_split(X6, Y6, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train6.shape)
print("Shape of test data:", X_test6.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2048)
Shape of test data: (899, 2048)


In [35]:
clf6 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models6, predictions6 = clf6.fit(X_train6, X_test6, Y_train6, Y_test6)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:26<00:19,  6.41s/it]

[16:56:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:29<00:00,  3.07s/it]


In [36]:
predictions6.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.84,0.84,0.84,0.84,1.74
RandomForestClassifier,0.83,0.83,0.83,0.83,1.45
LGBMClassifier,0.83,0.83,0.83,0.83,0.52
XGBClassifier,0.83,0.83,0.83,0.83,1.86
BaggingClassifier,0.83,0.83,0.83,0.83,2.39
SVC,0.82,0.82,0.82,0.82,17.43
NuSVC,0.82,0.82,0.82,0.82,19.05
DecisionTreeClassifier,0.81,0.81,0.81,0.81,0.78
KNeighborsClassifier,0.80,0.80,0.80,0.80,5.57


In [37]:
predictions6.to_csv('LazyPredict_MorganF_2048.csv')

In [38]:
#2048 bits MAP4 fingerprints
X7 = np.array(list((df2['fp_MAP4']))).astype(float)
#X.shape
Y7 = df2["activity"].values
#y.shape

In [39]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train7, X_test7, Y_train7, Y_test7 = train_test_split(X7, Y7, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train7.shape)
print("Shape of test data:", X_test7.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2048)
Shape of test data: (899, 2048)


In [40]:
# Defines and builds the lazyclassifier
clf7 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models7, predictions7 = clf7.fit(X_train7, X_test7, Y_train7, Y_test7)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [02:14<00:20,  6.90s/it]

[16:58:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [02:25<00:00,  5.01s/it]


In [41]:
predictions7.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.84,0.84,0.84,0.84,4.60
LGBMClassifier,0.84,0.84,0.84,0.84,5.86
RandomForestClassifier,0.83,0.83,0.83,0.83,5.64
ExtraTreesClassifier,0.83,0.83,0.83,0.83,1.69
NuSVC,0.82,0.81,0.81,0.82,19.10
SVC,0.80,0.80,0.80,0.80,18.07
KNeighborsClassifier,0.80,0.79,0.79,0.80,6.15
BaggingClassifier,0.80,0.80,0.80,0.80,26.67
AdaBoostClassifier,0.78,0.78,0.78,0.78,16.71


In [42]:
predictions7.to_csv('LazyPredict_MAP4_2048.csv')

### By combining 1D descriptors - 9 of them -  with fingerprints of 1024 bits

In [43]:
fp = np.array(list((df1['fp_MACCS']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X8 = np.column_stack((fp,oneD))
Y8 = df1["activity"].values
X8.shape

(4492, 176)

In [44]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train8, X_test8, Y_train8, Y_test8 = train_test_split(X8, Y8, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train8.shape)
print("Shape of test data:", X_test8.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 176)
Shape of test data: (899, 176)


In [45]:
# Defines and builds the lazyclassifier
clf8 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models8, predictions8 = clf8.fit(X_train8, X_test8, Y_train8, Y_test8)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:09<00:01,  2.12it/s]

[16:59:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.78it/s]


In [46]:
predictions8.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.82,0.82,0.82,0.82,0.36
NuSVC,0.82,0.82,0.82,0.82,1.55
RandomForestClassifier,0.81,0.81,0.81,0.81,0.35
LGBMClassifier,0.80,0.80,0.80,0.80,0.14
BaggingClassifier,0.80,0.80,0.80,0.80,0.24
SVC,0.80,0.80,0.80,0.80,1.39
XGBClassifier,0.80,0.80,0.80,0.80,0.38
KNeighborsClassifier,0.80,0.79,0.79,0.79,0.48
ExtraTreeClassifier,0.76,0.76,0.76,0.76,0.02


In [47]:
predictions8.to_csv('LazyPredict_MACCS_1Dadded.csv')

In [48]:
#with Morgan 1024 bits fingerprints
fp = np.array(list((df1['fp_Morgan3']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X9 = np.column_stack((fp,oneD))
Y9 = df1["activity"].values
X9.shape

(4492, 1033)

In [49]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train9, X_test9, Y_train9, Y_test9 = train_test_split(X9, Y9, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train9.shape)
print("Shape of test data:", X_test9.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1033)
Shape of test data: (899, 1033)


In [50]:
clf9 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models9, predictions9 = clf9.fit(X_train9, X_test9, Y_train9, Y_test9)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:38<00:08,  2.95s/it]

[16:59:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:39<00:00,  1.36s/it]


In [51]:
predictions9.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.83,0.83,0.83,0.83,0.60
ExtraTreesClassifier,0.83,0.83,0.83,0.83,0.77
XGBClassifier,0.83,0.83,0.83,0.83,0.91
NuSVC,0.83,0.82,0.82,0.83,9.20
LGBMClassifier,0.82,0.82,0.82,0.82,0.38
SVC,0.82,0.81,0.81,0.82,8.29
BaggingClassifier,0.81,0.81,0.81,0.81,1.28
QuadraticDiscriminantAnalysis,0.80,0.79,0.79,0.80,1.00
LinearDiscriminantAnalysis,0.79,0.79,0.79,0.79,0.68


In [52]:
predictions9.to_csv('LazyPredict_Morgan3_1Dadded.csv')

In [53]:
#with Morgan 1024 bits fingerprints with feautures
fp = np.array(list((df1['fp_MorganF']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X10 = np.column_stack((fp,oneD))
Y10 = df1["activity"].values
X10.shape

(4492, 1033)

In [54]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train10, X_test10, Y_train10, Y_test10 = train_test_split(X10, Y10, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train10.shape)
print("Shape of test data:", X_test10.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1033)
Shape of test data: (899, 1033)


In [55]:
clf10 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models10, predictions10 = clf10.fit(X_train10, X_test10, Y_train10, Y_test10)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:37<00:08,  2.86s/it]

[17:00:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:39<00:00,  1.35s/it]


In [56]:
predictions10.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.84,0.83,0.83,0.84,0.59
ExtraTreesClassifier,0.84,0.83,0.83,0.84,0.69
NuSVC,0.83,0.82,0.82,0.83,9.03
LGBMClassifier,0.82,0.82,0.82,0.82,0.49
XGBClassifier,0.82,0.82,0.82,0.82,1.09
SVC,0.82,0.82,0.82,0.82,8.06
BaggingClassifier,0.82,0.82,0.82,0.82,1.17
QuadraticDiscriminantAnalysis,0.80,0.79,0.79,0.79,1.00
KNeighborsClassifier,0.78,0.78,0.78,0.78,2.86


In [57]:
predictions10.to_csv('LazyPredict_MorganF_1Dadded.csv')

In [58]:
fp = np.array(list((df1['fp_MAP4']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X11 = np.column_stack((fp,oneD))
Y11 = df1["activity"].values
X11.shape

(4492, 1033)

In [59]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train11, X_test11, Y_train11, Y_test11 = train_test_split(X11, Y11, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train11.shape)
print("Shape of test data:", X_test11.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 1033)
Shape of test data: (899, 1033)


In [60]:
clf11 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models11, predictions11 = clf11.fit(X_train11, X_test11, Y_train11, Y_test11)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:04<00:09,  3.19s/it]

[17:01:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:08<00:00,  2.37s/it]


In [61]:
predictions11.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.83,0.83,0.83,0.83,3.84
LGBMClassifier,0.83,0.82,0.82,0.83,2.59
XGBClassifier,0.82,0.82,0.82,0.82,2.00
ExtraTreesClassifier,0.81,0.81,0.81,0.81,0.86
KNeighborsClassifier,0.81,0.80,0.80,0.81,2.96
NuSVC,0.79,0.79,0.79,0.79,9.30
SVC,0.79,0.78,0.78,0.79,8.41
BaggingClassifier,0.77,0.77,0.77,0.77,13.73
QuadraticDiscriminantAnalysis,0.76,0.76,0.76,0.76,1.01


In [62]:
predictions11.to_csv('LazyPredict_MAP4_1Dadded.csv')

### By combining 1D descriptors - 9 of them -  with fingerprints of 2048 bits

In [63]:
#with Morgan 2048 bits fingerprints
fp = np.array(list((df2['fp_Morgan3']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X12 = np.column_stack((fp,oneD))
Y12 = df2["activity"].values
X12.shape

(4492, 2057)

In [64]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train12, X_test12, Y_train12, Y_test12 = train_test_split(X12, Y12, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train12.shape)
print("Shape of test data:", X_test12.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2057)
Shape of test data: (899, 2057)


In [65]:
clf12 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models12, predictions12 = clf12.fit(X_train12, X_test12, Y_train12, Y_test12)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:24<00:19,  6.57s/it]

[17:02:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:27<00:00,  3.02s/it]


In [66]:
predictions12.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.87,0.87,0.87,0.87,1.12
RandomForestClassifier,0.85,0.85,0.85,0.85,0.90
LGBMClassifier,0.85,0.84,0.84,0.85,0.96
XGBClassifier,0.84,0.84,0.84,0.84,2.04
BaggingClassifier,0.83,0.84,0.84,0.83,2.19
NuSVC,0.83,0.82,0.82,0.83,19.93
SVC,0.82,0.82,0.82,0.82,18.07
ExtraTreeClassifier,0.80,0.80,0.80,0.80,0.16
CalibratedClassifierCV,0.79,0.79,0.79,0.79,17.39


In [67]:
predictions12.to_csv('LazyPredict_Morgan3_2048_1Dadded.csv')

In [68]:
#with Morgan 2048 bits fingerprints with feautures
fp = np.array(list((df2['fp_MorganF']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X13 = np.column_stack((fp,oneD))
Y13 = df2["activity"].values
X13.shape

(4492, 2057)

In [69]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train13, X_test13, Y_train13, Y_test13 = train_test_split(X13, Y13, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train13.shape)
print("Shape of test data:", X_test13.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2057)
Shape of test data: (899, 2057)


In [70]:
clf13 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models13, predictions13 = clf13.fit(X_train13, X_test13, Y_train13, Y_test13)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:22<00:18,  6.22s/it]

[17:04:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:26<00:00,  2.98s/it]


In [71]:
predictions13.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.85,0.85,0.85,0.85,2.77
RandomForestClassifier,0.85,0.85,0.85,0.85,0.87
BaggingClassifier,0.84,0.84,0.84,0.84,2.10
ExtraTreesClassifier,0.84,0.84,0.84,0.84,1.05
LGBMClassifier,0.83,0.84,0.84,0.83,0.58
NuSVC,0.82,0.82,0.82,0.82,19.01
SVC,0.82,0.82,0.82,0.82,17.01
DecisionTreeClassifier,0.81,0.81,0.81,0.81,0.44
KNeighborsClassifier,0.80,0.80,0.80,0.80,5.60


In [72]:
predictions13.to_csv('LazyPredict_MorganF_2048_1Dadded.csv')

In [73]:
#with MAP4 2048 bits fingerprints
fp = np.array(list((df2['fp_MAP4']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X14 = np.column_stack((fp,oneD))
Y14 = df2["activity"].values
X14.shape

(4492, 2057)

In [74]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train14, X_test14, Y_train14, Y_test14 = train_test_split(X14, Y14, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train14.shape)
print("Shape of test data:", X_test14.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (3593, 2057)
Shape of test data: (899, 2057)


In [75]:
clf14 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models14, predictions14 = clf14.fit(X_train14, X_test14, Y_train14, Y_test14)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [02:11<00:20,  6.92s/it]

[17:06:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [02:22<00:00,  4.92s/it]


In [76]:
predictions14.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.84,0.84,0.84,0.84,6.72
XGBClassifier,0.84,0.84,0.84,0.84,3.88
RandomForestClassifier,0.83,0.83,0.83,0.83,5.38
ExtraTreesClassifier,0.83,0.83,0.83,0.83,1.30
NuSVC,0.82,0.82,0.82,0.82,19.17
KNeighborsClassifier,0.81,0.80,0.80,0.80,6.14
SVC,0.80,0.80,0.80,0.80,18.17
BaggingClassifier,0.80,0.80,0.80,0.80,26.14
AdaBoostClassifier,0.78,0.78,0.78,0.78,15.45


In [77]:
predictions14.to_csv('LazyPredict_MAP4_2048_1Dadded.csv')